In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import sys

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Recommender System

In [3]:
# upload .csv files and set 'Product' column as the index for each dataframe

adverse_events_cosmetics = pd.read_csv('./datasets/adverse_events_cosmetics.csv').set_index('Product')
adverse_events_food = pd.read_csv('./datasets/adverse_events_food.csv').set_index('Product')
adverse_events_supplements = pd.read_csv('./datasets/adverse_events_supplements.csv').set_index('Product')

In [4]:
cv = CountVectorizer()
cv_matrix_cosmetics = cv.fit_transform(adverse_events_cosmetics['MedDRA Preferred Terms'])

cosine_sim_cosmetics = cosine_similarity(cv_matrix_cosmetics, cv_matrix_cosmetics)

In [5]:
cv_matrix_supplements = cv.fit_transform(adverse_events_supplements['MedDRA Preferred Terms'])

cosine_sim_supplements = cosine_similarity(cv_matrix_supplements, cv_matrix_supplements)

In [6]:
cv_matrix_food = cv.fit_transform(adverse_events_food['MedDRA Preferred Terms'])

cosine_sim_food = cosine_similarity(cv_matrix_food, cv_matrix_food)

In [7]:
# ref: https://towardsdatascience.com/how-to-build-from-scratch-a-content-based-movie-recommender-with-natural-language-processing-25ad400eb243

# function that will take in a product and output said product along with 10 other products that are most related
    #  or reference similar symptoms
def beware_recommendations(df, product, cosine_sim):

    indices = pd.Series(df.index)
    beware_of_products = []
    
    # pulls index of the product that is inputted into this function
    # prod_index = indices[indices == product].index[0] >>> this is if I want function to take exact product name
    try:
        prod_index = indices[indices.str.contains(product)].index[0]

        # want df output to also include the inputted product for comparison
        beware_of_products.append(product)

        # puts similarity scores in descending order
        similarity_scores = pd.Series(cosine_sim[prod_index]).sort_values(ascending = False)

        # pulls index of 10 products that have similar reported adverse events
        similar_10_prod = list(similarity_scores.iloc[1:11].index)

        # adding list of products to beware_of_products list
        for i in similar_10_prod:
            beware_of_products.append(list(df.index)[i])

        # want to output a dataframe that will show the products with associated descriptions and details of adverse events  
        return df.loc[beware_of_products]
    
    except:
        print("Great news! This product hasn't been reported to the FDA as causing negative side effects.")




In [8]:
beware_recommendations(adverse_events_cosmetics, 'foundation', cosine_sim_cosmetics)

,Description,MedDRA Preferred Terms
Product,,
foundation,Cosmetics,"RASH, ERYTHEMA"
devacurl nopoo decadence,Cosmetics,"PRURITUS, HYPERSENSITIVITY, BURNING SENSATION"
avas cosmetics total coverage foundation,Cosmetics,"PRURITUS, HYPERSENSITIVITY, BURNING SENSATION"
clairol herbal essence shampoo for dry and damaged,Cosmetics,"RASH, PRURITUS, HYPERSENSITIVITY, BURNING SENS..."
fair and lovely advanced multivitamin cream,Cosmetics,"PRURITUS, BURNING SENSATION"
almay one coat nourishing mascara black,Cosmetics,"HYPERSENSITIVITY, BURNING SENSATION"
mane and tail hair and body mist,Cosmetics,"HYPERSENSITIVITY, BURNING SENSATION"
serious skin care reversetrol drench double phase cleansing fusion,Cosmetics,"HYPERSENSITIVITY, BURNING SENSATION"
st ives apricot scrub facial cleanser,Cosmetics,"HYPERSENSITIVITY, BURNING SENSATION"


In [9]:
beware_recommendations(adverse_events_food, 'rubinoff', cosine_sim_food)

Great news! This product hasn't been reported to the FDA as causing negative side effects.


In [10]:
# beware_recommendations(adverse_events_supplements, '', cosine_sim_supplements)

## Detailed dataframe for research

In [11]:
adverse_events = pd.read_csv('./datasets/adverse_events_2.csv')

In [12]:
adverse_events['CAERS Created Date'] = pd.to_datetime(adverse_events['CAERS Created Date']) 
adverse_events['Date of Event'] = pd.to_datetime(adverse_events['Date of Event'])

In [13]:
prod = adverse_events[adverse_events['Product'].str.contains('LIMBREL')]
prod.sort_values('Date of Event')

,Report ID,CAERS Created Date,Date of Event,Product Type,Product,Product Code,Description,Patient Age,Age Units,Sex,MedDRA Preferred Terms,Outcomes
7942,92012,2007-03-14,2006-08-28,SUSPECT,LIMBREL 250MG,41,Dietary Conventional Foods/Meal Replacements,65.0,year(s),F,LIVER FUNCTION TEST ABNORMAL,"Patient Visited Healthcare Provider, Medically..."
7937,92013,2007-03-14,2006-09-18,SUSPECT,LIMBREL 500MG,41G,"G Medical Foods, N.E.C.",75.0,year(s),F,"VASCULITIS, RASH, DYSPNOEA, CHRONIC OBSTRUCTIV...",Hospitalization
7938,92013,2007-03-14,2006-09-18,SUSPECT,LIMBREL 250MG,41G,"G Medical Foods, N.E.C.",75.0,year(s),F,"VASCULITIS, RASH, DYSPNOEA, CHRONIC OBSTRUCTIV...",Hospitalization
9772,97202,2007-10-16,2007-07-01,SUSPECT,LIMBREL,41,Dietary Conventional Foods/Meal Replacements,NaN,NaN,NaN,BLOOD ELECTROLYTES DECREASED,"Patient Visited Healthcare Provider, Medically..."
14068,107806,2008-11-12,2008-06-23,SUSPECT,LIMBREL 500 MG,41,Dietary Conventional Foods/Meal Replacements,56.0,year(s),F,"SWELLING, HEADACHE, BLOOD PRESSURE ABNORMAL",Medically Important
14039,107678,2008-11-06,2008-08-02,SUSPECT,LIMBREL CAPS 250MG,41,Dietary Conventional Foods/Meal Replacements,NaN,NaN,NaN,"NAUSEA, MALAISE, HEADACHE",Medically Important
13670,106798,2008-10-02,2008-08-20,SUSPECT,LIMBREL 500 MG CAPSULES,41,Dietary Conventional Foods/Meal Replacements,NaN,NaN,NaN,"PROTHROMBIN TIME SHORTENED, MUCOUS MEMBRANE DI...","Life Threatening, Patient Visited Healthcare P..."
15768,111599,2009-03-30,2009-03-02,SUSPECT,LIMBREL (FLAVOCOXID) 250 MG,41,Dietary Conventional Foods/Meal Replacements,NaN,NaN,NaN,"URTICARIA, RASH, DIZZINESS",Medically Important
15514,110955,2009-03-10,2009-03-05,SUSPECT,LIMBREL 500 MG,41,Dietary Conventional Foods/Meal Replacements,66.0,year(s),F,"PALPITATIONS, PAIN, FATIGUE, FATIGUE, CHILLS, ...",Medically Important
17886,116078,2009-07-27,2009-03-31,SUSPECT,LIMBREL 500MG,41,Dietary Conventional Foods/Meal Replacements,52.0,year(s),M,"THROMBOSIS, SWELLING, PYREXIA, NIGHT SWEATS, M...","Life Threatening, Hospitalization, Patient Vis..."


In [14]:
adverse_events[(adverse_events['CAERS Created Date'] > '2007-1-1') & (adverse_events['CAERS Created Date'] < '2017-11-9') &
              adverse_events['Product'].str.contains('LIMBREL')]

,Report ID,CAERS Created Date,Date of Event,Product Type,Product,Product Code,Description,Patient Age,Age Units,Sex,MedDRA Preferred Terms,Outcomes
7937,92013,2007-03-14,2006-09-18,SUSPECT,LIMBREL 500MG,41G,"G Medical Foods, N.E.C.",75.0,year(s),F,"VASCULITIS, RASH, DYSPNOEA, CHRONIC OBSTRUCTIV...",Hospitalization
7938,92013,2007-03-14,2006-09-18,SUSPECT,LIMBREL 250MG,41G,"G Medical Foods, N.E.C.",75.0,year(s),F,"VASCULITIS, RASH, DYSPNOEA, CHRONIC OBSTRUCTIV...",Hospitalization
7942,92012,2007-03-14,2006-08-28,SUSPECT,LIMBREL 250MG,41,Dietary Conventional Foods/Meal Replacements,65.0,year(s),F,LIVER FUNCTION TEST ABNORMAL,"Patient Visited Healthcare Provider, Medically..."
9772,97202,2007-10-16,2007-07-01,SUSPECT,LIMBREL,41,Dietary Conventional Foods/Meal Replacements,NaN,NaN,NaN,BLOOD ELECTROLYTES DECREASED,"Patient Visited Healthcare Provider, Medically..."
9840,97443,2007-10-24,NaT,SUSPECT,LIMBREL 250MG,41,Dietary Conventional Foods/Meal Replacements,70.0,year(s),F,BODY TEMPERATURE INCREASED,Medically Important
11268,101362,2008-03-19,NaT,SUSPECT,LIMBREL 250 MG,41,Dietary Conventional Foods/Meal Replacements,61.0,year(s),F,"WEIGHT DECREASED, TREMOR, RESPIRATORY DISTRESS...","Patient Visited Healthcare Provider, Medically..."
13670,106798,2008-10-02,2008-08-20,SUSPECT,LIMBREL 500 MG CAPSULES,41,Dietary Conventional Foods/Meal Replacements,NaN,NaN,NaN,"PROTHROMBIN TIME SHORTENED, MUCOUS MEMBRANE DI...","Life Threatening, Patient Visited Healthcare P..."
14039,107678,2008-11-06,2008-08-02,SUSPECT,LIMBREL CAPS 250MG,41,Dietary Conventional Foods/Meal Replacements,NaN,NaN,NaN,"NAUSEA, MALAISE, HEADACHE",Medically Important
14068,107806,2008-11-12,2008-06-23,SUSPECT,LIMBREL 500 MG,41,Dietary Conventional Foods/Meal Replacements,56.0,year(s),F,"SWELLING, HEADACHE, BLOOD PRESSURE ABNORMAL",Medically Important
15514,110955,2009-03-10,2009-03-05,SUSPECT,LIMBREL 500 MG,41,Dietary Conventional Foods/Meal Replacements,66.0,year(s),F,"PALPITATIONS, PAIN, FATIGUE, FATIGUE, CHILLS, ...",Medically Important
